# Análisis económico | Extractos bancarios

Github realiza un renderizado estático de los notebook y no incluye el HTML/JavaScript incrustado que compone un gráfico plotly, para un mejor visualización: https://nbviewer.org/github/pachums/analisis_bancario/blob/main/analisis_extractos_tutorial.ipynb

Script para el análisis económico de los extractos bancarios, previo a su ejecución se ha de actualizar el apartado 1 y añadir en la carpeta junto con el script el extractos.


**1. Estado actual**
    
**2. Procesamiento de datos**
    
**3. Análisis económico**

    3.1. Últimos 4 meses
    3.2. Mes de estudio y cuatrimestre en categorías
    3.3. Anual en categorías

In [14]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import dateutil.relativedelta
from os import listdir
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import warnings

sns.set_style('darkgrid')
warnings.filterwarnings("ignore")

## 1. Estado actual

Analizamos la cantidad actual de dinero y dónde se ubica, asimismo como una evolución mensual de los ahorros.

In [15]:
data = [['Banco', 3000], 
        ['Inversiones', 2000], 
        ['PayPal', 400],
        ['Préstamo', 1000]]
  
 # creamos un dataframe
df_economia = pd.DataFrame(data, columns = ['Activo', 'Cantidad'])
print(f"TOTAL: {df_economia['Cantidad'].sum()} €")

fig = px.pie(values=list(df_economia['Cantidad']), names=list(df_economia['Activo']), title='Distribución actual')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

TOTAL: 6400 €


In [16]:
 # al finalizar cada mes actualizamos el capital total
mes = [3,4,5,6,7,8,9,10,11]
ahorro = [3900,4850,4050,4200,4900,5200,6700,7290,8047]

 # creamos dataframe y representamos
data = {'Mes' : mes, 'Ahorro' : ahorro}
df_data = pd.DataFrame(data)
fig = px.line(df_data, x='Mes', y='Ahorro', text="Ahorro", title='Evolución mensual de ahorros')
fig.update_traces(textposition='top center')
fig.update_layout(yaxis={"range":[0,20000]},margin={"t":50,"b":0},height=550)

fig.show()

## 2. Procesamiento de datos

Procesamos el extracto bancario de los movimientos, limpiamos el dataframe y seleccionamos la información relevante, creando alguna columna más para su posterior análisis y representación.

In [17]:
 # automatizamos la lectura del extracto
for file in listdir(os.getcwd()):
    if 'extracto' in file:
        df_ruta = file

 # cargamos datos
df = pd.read_csv(df_ruta, sep=",", decimal=".")
df.head(5)

,Fecha ctble,Fecha valor,Concepto,Importe,Moneda,Saldo,Moneda.1,Concepto ampliado
0,2021-01-04,02/01/2021,434001032209 DECATHLON LUGONES \LUGONES\ES210...,-10.0,EUR,2010.0,EUR,NaN
1,2021-01-04,02/01/2021,434001032209 FNAC PRINCIPAD\CASTANERO (S\ES210...,-40.0,EUR,1970.0,EUR,NaN
2,2021-01-04,02/01/2021,434001032209 MEDIA MARKT - \VILLAR (P. S\ES210...,-53.0,EUR,1917.0,EUR,NaN
3,2021-01-04,02/01/2021,434001032209 SIERO RESTAURACION S\VIELLA\ES210...,-4.0,EUR,1913.0,EUR,NaN
4,2021-01-04,04/01/2021,"71731918R PayPal (Europe) S.a.r.l. et Cie., S....",200.0,EUR,2113.0,EUR,YYW1011772843930


In [18]:
 # eliminamos columnas irrelevantes
df.drop(['Fecha valor', 'Moneda', 'Moneda.1', 'Concepto ampliado', 'Saldo'], axis=1, inplace=True)

 # renombramos y cambiamos el formato a datatime
df.rename(columns={"Fecha ctble" : "Fecha", "Concepto" : "Desc"},inplace=True)
df['Fecha'] = pd.to_datetime(df['Fecha'])

 # añadimos tipo: ingreso o gasto
df['Tipo']=df['Importe'].apply(lambda x: "ingreso" if x>0 else "gasto")

 # eliminamos traspasos internos
df.drop(df[df['Desc'].str.contains("TRASPASO AUTOMATICO")].index, inplace=True)

df.head(3)

,Fecha,Desc,Importe,Tipo
0,2021-01-04,434001032209 DECATHLON LUGONES \LUGONES\ES210...,-10.0,gasto
1,2021-01-04,434001032209 FNAC PRINCIPAD\CASTANERO (S\ES210...,-40.0,gasto
2,2021-01-04,434001032209 MEDIA MARKT - \VILLAR (P. S\ES210...,-53.0,gasto


Creamos categorias: en base a palabras contenidas en las descripciones de los registros los clasificamos en las distintas catergorias.

In [19]:
supermercado = ['alimerka', 'toogoodtog', 'mercadona', 'masymas', 'carrefour', 'aldi', 'fruteria', 'hiper', 'alimen']
servicios = ['farmacia', 'brico', 'leroy', 'ferreteria', 'desguaces', 'foto', 'honghezhou', 'yi xiang', 'serv. tribut.', 'LA CAIXA', 'reintegro'] 
transporte = ['station', 'ballenoil', 'combuscan', 'alsa', 'blabla', 'gasol', 'metro', 'beroil', 'eess leon', 'petro',
             'easygas', 'chofa', 'e.servicio vill', 'park']
deportes = ['decathlon', 'boulder', 'one move', 'blackisard', 'cervantes', 'd_ruta', 'esqui', 'fuentes de i', 'refugio', 'montana']
restaurantes = ['bar', 'restaurante', 'cafe', 'helados', 'hamburgues', 'chocolat', 'hosteleria', 'pizzeria', 'pizza',
                'kebab', 'doner', 'el puerto', 'pindal', 'quesu', 'dos de azucar', 'sarralde', 'autoking', 'horchateria', 'bocata']
compra_internet = ['amzn mktp', 'amazon', 'wallapop', 'aliexpress']
recurrente = ['digi','medicos sin fronteras', 'vodafone', 'simyo', 'couchsurfing', 'amazon prime']
#alquiler

def agrupador(desc):
    desc = desc.lower() 

    if any(x in desc for x in supermercado):
        categoria = 'supermercado'
    elif any(x in desc for x in servicios):
        categoria = 'servicios'
    elif any(x in desc for x in transporte):
        categoria = 'transporte'
    elif any(x in desc for x in deportes):
        categoria = 'deportes'
    elif any(x in desc for x in restaurantes):
        categoria = 'restaurantes'
    elif any(x in desc for x in compra_internet):
        categoria = 'compra_internet'
    elif any(x in desc for x in recurrente):
        categoria = 'recurrente'
    else:
        categoria = 'OTROS'
        
    return categoria

#### Filtros específicos y fechas

In [20]:
 # aplicamos la funcion agrupador
df['Categoria'] = df['Desc'].apply(agrupador)

 # todos los importes positivos y redondeados
df['Importe'] = abs(df['Importe'])
df['Importe'] = round(df['Importe'],2)

 # separamos por meses y años
anno21 = df[df['Fecha'].dt.year == 2021]
anno22 = df[df['Fecha'].dt.year == 2022]

sep21 = df[(df['Fecha'].dt.year == 2021) & (df['Fecha'].dt.month == 9)]
oct21 = df[(df['Fecha'].dt.year == 2021) & (df['Fecha'].dt.month == 10)]
nov21 = df[(df['Fecha'].dt.year == 2021) & (df['Fecha'].dt.month == 11)]
dic21 = df[(df['Fecha'].dt.year == 2021) & (df['Fecha'].dt.month == 12)]
ene22 = df[(df['Fecha'].dt.year == 2022) & (df['Fecha'].dt.month == 1)]
feb22 = df[(df['Fecha'].dt.year == 2022) & (df['Fecha'].dt.month == 2)]
mar22 = df[(df['Fecha'].dt.year == 2022) & (df['Fecha'].dt.month == 3)]
abr22 = df[(df['Fecha'].dt.year == 2022) & (df['Fecha'].dt.month == 4)]


 # lista de meses desde sep21 hasta actual
lista_meses = [sep21, oct21, nov21, dic21, ene22, feb22, mar22, abr22]
lista_meses = lista_meses[0:4+int(date.today().strftime("%m"))]

 # último cuatrimestre
df_cuatri = df[((df['Fecha'].dt.year == int((date.today() - dateutil.relativedelta.relativedelta(months=3)).strftime("%Y"))) 
                & (df['Fecha'].dt.month == int((date.today() - dateutil.relativedelta.relativedelta(months=3)).strftime("%m")))) |
              ((df['Fecha'].dt.year == int((date.today() - dateutil.relativedelta.relativedelta(months=2)).strftime("%Y"))) 
                & (df['Fecha'].dt.month == int((date.today() - dateutil.relativedelta.relativedelta(months=2)).strftime("%m")))) |
              ((df['Fecha'].dt.year == int((date.today() - dateutil.relativedelta.relativedelta(months=1)).strftime("%Y"))) 
                & (df['Fecha'].dt.month == int((date.today() - dateutil.relativedelta.relativedelta(months=1)).strftime("%m")))) |
              ((df['Fecha'].dt.year == int(date.today().strftime("%Y"))) 
               & (df['Fecha'].dt.month == int(date.today().strftime("%m"))))]

 # mes actual
df_act = lista_meses[-1]

In [25]:
df.loc[53:60]

,Fecha,Desc,Importe,Tipo,Categoria
53,2021-01-26,434001032209 MERCADONA EL FONTAN \OVIEDO\ES210...,30.77,gasto,supermercado
54,2021-01-27,269730430730 SIMYO,8.00,gasto,recurrente
55,2021-01-27,ABONO ABANCAPAY: 55640765-4086-4799-b156,4.00,ingreso,OTROS
56,2021-01-28,434001032209 ALIEXPRESS.COM Luxembourg 265670002,50.63,gasto,compra_internet
57,2021-01-28,71731918R GRUPO DE MONTANA TORREBLANCA,15.00,ingreso,deportes
58,2021-01-28,"00071731918R FUNDACION MATRIX, INVESTIG.Y DESA...",1383.97,ingreso,OTROS
59,2021-01-28,434001032209 LIBRERIA LA PALMA \OVIEDO\ES210...,18.00,gasto,OTROS
60,2021-01-29,434001032209 LIBERBANK \OVIEDO\ES210...,300.00,gasto,OTROS


## 3. Análisis económico

### 3.1. Últimos 4 meses 

In [8]:
 # obtenemos últimos 4 meses de lista_meses
meses = [lista_meses[-4].iloc[0,0].month, lista_meses[-3].iloc[0,0].month, 
         lista_meses[-2].iloc[0,0].month, lista_meses[-1].iloc[0,0].month]
 
 # creamos las variables para el plot
y_ingreso = [lista_meses[-4][lista_meses[-4]['Tipo']=='ingreso']['Importe'].sum(),
             lista_meses[-3][lista_meses[-3]['Tipo']=='ingreso']['Importe'].sum(),
             lista_meses[-2][lista_meses[-2]['Tipo']=='ingreso']['Importe'].sum(),
             lista_meses[-1][lista_meses[-1]['Tipo']=='ingreso']['Importe'].sum()]
y_gasto = [lista_meses[-4][lista_meses[-4]['Tipo']=='gasto']['Importe'].sum(),
           lista_meses[-3][lista_meses[-3]['Tipo']=='gasto']['Importe'].sum(),
           lista_meses[-2][lista_meses[-2]['Tipo']=='gasto']['Importe'].sum(),
           lista_meses[-1][lista_meses[-1]['Tipo']=='gasto']['Importe'].sum()]
y_ahorro = [lista_meses[-4][lista_meses[-4]['Tipo']=='ingreso']['Importe'].sum() - lista_meses[-4][lista_meses[-4]['Tipo']=='gasto']['Importe'].sum(),
            lista_meses[-3][lista_meses[-3]['Tipo']=='ingreso']['Importe'].sum() - lista_meses[-3][lista_meses[-3]['Tipo']=='gasto']['Importe'].sum(),
            lista_meses[-2][lista_meses[-2]['Tipo']=='ingreso']['Importe'].sum() - lista_meses[-2][lista_meses[-2]['Tipo']=='gasto']['Importe'].sum(), 
            lista_meses[-1][lista_meses[-1]['Tipo']=='ingreso']['Importe'].sum() - lista_meses[-1][lista_meses[-1]['Tipo']=='gasto']['Importe'].sum() if round(lista_meses[-1][lista_meses[-1]['Tipo']=='ingreso']['Importe'].sum() - lista_meses[-1][lista_meses[-1]['Tipo']=='gasto']['Importe'].sum(), 1) > 0 else 0]

 # creamos el plot
fig = go.Figure(data=[
    go.Bar(name = 'ingreso', x = meses, y = y_ingreso, text = y_ingreso, textposition = 'outside'),
    go.Bar(name = 'gasto', x = meses, y = y_gasto, text = y_gasto, textposition = 'outside'),
    go.Bar(name = 'ahorro', x = meses, y = y_ahorro, text = y_ahorro, textposition = 'outside')
])

 # ajustes y show
fig.update_layout(barmode='group', title='Últimos 4 meses')
fig.update_traces(texttemplate='%{text:.1f}')

fig.show()

### 3.2. Mes de estudio en categorías

In [26]:
fig = px.bar(df_act[df_act['Tipo']=='gasto'].groupby(['Categoria'], as_index = False).sum(),
             x='Categoria', y='Importe', title='Mes de estudio en categorías', color='Categoria', text='Importe')

fig.update_layout(xaxis={'categoryarray' : ['alquiler', 'servicios', 'supermercado', 'deportes']})
fig.update_traces(texttemplate='%{text:.2f}')

fig.show()

In [10]:
fig = px.bar(df_cuatri[df_cuatri['Tipo']=='gasto'].groupby(['Categoria'], as_index = False).sum(),
             x='Categoria', y='Importe', title='Último cuatrimestre', color='Categoria', text='Importe')
fig.update_layout(xaxis={'categoryarray' : ['alquiler', 'servicios', 'supermercado', 'deportes']})
fig.update_traces(texttemplate='%{text:.2f}')

fig.show()

### 3.3. Anual en categorías

In [11]:
fig = px.bar(anno21[anno21['Tipo']=='gasto'].groupby(['Categoria'], as_index = False).sum(),
             x='Categoria', y='Importe', title='Gastos anuales 2021', color='Categoria')
fig.update_layout(xaxis={'categoryarray' : ['alquiler', 'servicios', 'supermercado', 'deportes']})

fig.show()